# Libraries

In [1]:
import os
import cv2
import numpy as np
import time
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from PIL import Image
from sklearn.metrics import classification_report
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix

/Users/rey/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Global Var

In [2]:
IMAGE_SIZE = 64  
data = []
labels = []

# Pre-Processing and Loading

In [3]:
def prepareDataset(path):
    filePaths = []
    y = []
    count = 0
    name = ''

    for dirname, _, filenames in os.walk(path):
        if count != 0:
            x = dirname.split('/')[-1]
            index = x.rindex('_name_')
            name = x[index+6:].replace(' ','')
        for filename in filenames:
            full_path = os.path.join(dirname, filename)
            filePaths.append(full_path)
            y.append(name)
        
        count += 1
    return filePaths, y

In [4]:
hispanic_path = 'Selfies_ID_Images_dataset/11_sets_Hispanics'  
caucasian_path = 'Selfies_ID_Images_dataset/18_sets_Caucasians'  

hispanic_filePaths, hispanic_labels = prepareDataset(hispanic_path)
caucasian_filePaths, caucasian_labels = prepareDataset(caucasian_path)


filePaths = hispanic_filePaths + caucasian_filePaths
y_labels = hispanic_labels + caucasian_labels  

# Load and preprocess the images
for i, filePath in enumerate(filePaths):
    if not filePath.lower().endswith(('.png', '.jpg', '.jpeg')):
        print(f"Skipping non-image file: {filePath}")
        continue
    try:
        pil_image = Image.open(filePath).convert('RGB')
        image = np.array(pil_image)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        data.append(image)
        labels.append(y_labels[i])
    except (IOError, OSError) as e:
        print(f"Warning: Could not read image from {filePath} - {e}")
#
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [5]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

number_of_classes = labels.shape[1]
print("Classes Count: ",number_of_classes)
np.save('label_classes.npy', lb.classes_)

Classes Count:  11


# Model Building

In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(number_of_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-4), 
              metrics=['accuracy'])

# Model Training

In [7]:
model.fit(data, labels, epochs=50, batch_size=32)
model.save('Models/face_recognition_model.h5')

Epoch 1/50
6/6 [==============================] - 1s 49ms/step - loss: 2.4829 - accuracy: 0.1091
Epoch 2/50
6/6 [==============================] - 0s 47ms/step - loss: 2.4058 - accuracy: 0.1333
Epoch 3/50
6/6 [==============================] - 0s 46ms/step - loss: 2.3535 - accuracy: 0.1273
Epoch 4/50
6/6 [==============================] - 0s 47ms/step - loss: 2.3633 - accuracy: 0.1152
Epoch 5/50
6/6 [==============================] - 0s 48ms/step - loss: 2.2697 - accuracy: 0.2242
Epoch 6/50
6/6 [==============================] - 0s 48ms/step - loss: 2.2876 - accuracy: 0.2000
Epoch 7/50
6/6 [==============================] - 0s 44ms/step - loss: 2.2436 - accuracy: 0.2364
Epoch 8/50
6/6 [==============================] - 0s 51ms/step - loss: 2.1919 - accuracy: 0.2303
Epoch 9/50
6/6 [==============================] - 0s 49ms/step - loss: 2.1745 - accuracy: 0.2788
Epoch 10/50
6/6 [==============================] - 0s 47ms/step - loss: 2.1265 - accuracy: 0.3091
Epoch 11/50
6/6 [============

/Users/rey/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Evaluation

In [9]:
model = load_model('Models/face_recognition_model.h5')
predictions = model.predict(data)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(labels, axis=1)
lb.classes_ = np.load('label_classes.npy')
report = classification_report(true_classes, predicted_classes, target_names=lb.classes_)
print(report)

6/6 [==============================] - 0s 13ms/step
              precision    recall  f1-score   support

   Alejandra       1.00      0.93      0.97        15
       Bruno       1.00      0.60      0.75        15
      Daiane       0.74      0.93      0.82        15
    Fernanda       0.87      0.87      0.87        15
     Gabriel       1.00      0.93      0.97        15
     Juliana       0.92      0.73      0.81        15
        Luis       0.83      1.00      0.91        15
     Matheus       0.75      1.00      0.86        15
     Rayanne       0.93      0.93      0.93        15
     Rômulo       0.74      0.93      0.82        15
     Weslley       1.00      0.67      0.80        15

    accuracy                           0.87       165
   macro avg       0.89      0.87      0.86       165
weighted avg       0.89      0.87      0.86       165



In [10]:
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[14  0  1  0  0  0  0  0  0  0  0]
 [ 0  9  2  0  0  1  1  0  1  1  0]
 [ 0  0 14  0  0  0  0  0  0  1  0]
 [ 0  0  0 13  0  0  1  0  0  1  0]
 [ 0  0  1  0 14  0  0  0  0  0  0]
 [ 0  0  0  1  0 11  0  3  0  0  0]
 [ 0  0  0  0  0  0 15  0  0  0  0]
 [ 0  0  0  0  0  0  0 15  0  0  0]
 [ 0  0  0  0  0  0  0  0 14  1  0]
 [ 0  0  0  0  0  0  0  1  0 14  0]
 [ 0  0  1  1  0  0  1  1  0  1 10]]
